# How do I _copy_ an app from the Seven Bridges _Public Apps_ repository?
## Overview
Apps include both tools (individual bioinformatics utilities) and workflows (chains or pipelines of connected tools). Apps are located in one of two places: the [Seven Bridges Public Apps repository](https://igor.sbgenomics.com/public/apps) or a user-created project.

The Seven Bridges Platform offers a repository of [publicly available apps](https://igor.sbgenomics.com/public/apps) suitable for many different types of data analysis. Apps include both tools (individual bioinformatics utilities) and workflows (chains or pipelines of connected tools). The publicly available apps are maintained by the Seven Bridges Platform bioinformatics team to represent the latest tool versions.

## Objective

You can copy an app from the Public App repository or from a project of which you're a member. Here, we focus on **copying an app from the Public App repository**. [[reference](http://docs.sevenbridges.com/docs/copy-an-app)]
 

## Procedure
1. We'll list all our projects.
2. Then, we'll list the apps within the Seven Bridges Public Apps repository. 
3. Lastly, we'll copy an app from the Public Apps repository to the first project in our list of projects.

## Prerequisites
 1. You need to be a member (or owner) of at least one project. Learn more about creating a new project. [[recipe](projects_makeNew.ipynb)] [[reference](http://docs.sevenbridges.com/docs/create-a-new-project)].
 2. You need your **authentication token** and the API needs to know about it. See <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a> for details. Learn more about [obtaining your authentication token](http://docs.sevenbridges.com/v1.0/docs/get-your-authentication-token).
 3. You understand how to list projects of which you are a member. We will use this call directly to select a project below. [[recipe](projects_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-your-projects)]
 4. You understand how to list apps within a project. We will use this call below and select an app. [[recipe](apps_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-apps-available-to-you)]
 
Note that as with all okAPI recipes, this recipe makes use of the [Seven Bridges Public API Python library](../Tutorials/SBPLAT/Setup_API_environment.ipynb)
 
## Imports
We import the `Api` class from the official `sevenbridges-python` bindings below.  
We also import the `Conflict` error to handle situations when trying to copy over an existing app.

In [ ]:
import sevenbridges as sbg
from sevenbridges.errors import Conflict

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify credentials file profile {cgc, sbg, default}
prof = 'default'

config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy a Public App
First, we'll list all our projects. Then, we'll list the apps within the Public Reference project. Lastly, we'll copy an app from the Public Reference project to the my\_project. Here we also explicitly set _'limit':100_ inside the _query_. This helps speed up the auto-pagination feature within the object constructor.

The critical information for this `POST` request is the **app_id**. Note, you are **not** allowed to copy the same app **and** assign the same name<sup>1</sup> more than once. If you change the name, it is ok. 

To make these results very obvious, use an empty project as your my\_project (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or add the _name_ arguement to something ridiculous like 'Orange Mocha Frapachino Maker'. In this example, we will handle these situations with a predefined error `Conflict` for this situation.

<sup>1</sup> Note that setting the **name** of a copied app defines the app **id**, which must be unique on the platform. Saving over an existing ID will raise a `Conflict` error.

In [ ]:
# (optional) Print names of all Public Apps
public_app_names = [a.name for a in api.apps.query(visibility='public', limit=100).all()]
public_app_names.sort()

for app_name in public_app_names:
    print(app_name)

In [ ]:
# [USER INPUT] Set project and app names
# Note that you can have multiple apps or projects with the same name. It is best practice to reference entities by ID.
my_project_name = 'Project name'
a_name = 'GRAF Germline Variant Detection Workflow'

# pull out target project
my_project = api.projects.query(name=my_project_name)

# Double-check that target project exists
if not my_project:
    print('Target project {} not found, check spelling'.format(my_project_name))
    raise KeyboardInterrupt
else:
    
    my_project = my_project[0]
        
public_app = [a for a in api.apps.query(visibility='public', limit=100).all() 
                 if a.name == a_name][0]

try:
    new_app = public_app.copy(project=my_project)
    print('App {} copied to Project {}.'.format(public_app.name, my_project.name))
except Conflict:
    new_app = [a for a in api.apps.query(project=my_project) if a.name == a_name][0]
    print('App already exists in the destination project, reusing existing app.')
        
# re-list apps in target project to verify the copy worked
my_app_names = [a.name for a in
                api.apps.query(project=my_project.id, limit=100).all()]

if a_name in my_app_names:
    print('Sucessfully copied or reused one app!')
else:
    print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-an-app).